In [1]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model = "gpt-3.5-turbo", temperature= 0)

In [2]:
from langchain.agents import tool

@tool
def get_word_length(word:str) -> int:
    """Returns the length of a word."""
    return len(word)

tools = [get_word_length]

In [3]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system","You are a very powerful assistant, but don't know current events",
        ),
        ("user","{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad")
    ]
)

# 
How does the agent know what tools it can use?

In this case we’re relying on OpenAI tool calling LLMs, 

which take tools as a separate argument and have been specifically trained to know when to invoke those tools.

To pass in our tools to the agent, we just need to format them to the OpenAI tool format and pass them to our model.

(By bind-ing the functions, we’re making sure that they’re passed in each time the model is invoked.)



In [4]:
llm_with_tools = llm.bind_tools(tools)

In [5]:
from langchain.agents.format_scratchpad.openai_tools import (format_to_openai_tool_messages,)
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

agent = (
    {
        "input": lambda x:x["input"],
        "agent_scratchpad": lambda x:format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
    }
    |prompt
    |llm_with_tools
    | OpenAIToolsAgentOutputParser()
)

In [6]:
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent = agent, tools= tools , verbose=True)

In [8]:
list(agent_executor.stream({"input":"How many words are there in KatsukiBakugo"}))



> Entering new AgentExecutor chain...

Invoking: `get_word_length` with `{'word': 'KatsukiBakugo'}`


13There are 13 words in "KatsukiBakugo".

> Finished chain.


[{'actions': [ToolAgentAction(tool='get_word_length', tool_input={'word': 'KatsukiBakugo'}, log="\nInvoking: `get_word_length` with `{'word': 'KatsukiBakugo'}`\n\n\n", message_log=[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_pGSwislb85SrRISGqp8nChO1', 'function': {'arguments': '{"word":"KatsukiBakugo"}', 'name': 'get_word_length'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls'}, id='run-a679d7c5-f8a5-49a2-ab80-2655afd019e6', tool_calls=[{'name': 'get_word_length', 'args': {'word': 'KatsukiBakugo'}, 'id': 'call_pGSwislb85SrRISGqp8nChO1'}], tool_call_chunks=[{'name': 'get_word_length', 'args': '{"word":"KatsukiBakugo"}', 'id': 'call_pGSwislb85SrRISGqp8nChO1', 'index': 0}])], tool_call_id='call_pGSwislb85SrRISGqp8nChO1')],
  'messages': [AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_pGSwislb85SrRISGqp8nChO1', 'function': {'arguments': '{"word":"KatsukiBakugo"}', 'name': 'get_word_le

In [9]:
agent_executor.invoke({"input":"How many words are there in KatsukiBakugo"})



> Entering new AgentExecutor chain...

Invoking: `get_word_length` with `{'word': 'KatsukiBakugo'}`


13There are 13 words in "KatsukiBakugo".

> Finished chain.


{'input': 'How many words are there in KatsukiBakugo',
 'output': 'There are 13 words in "KatsukiBakugo".'}

## Adding Memory



In [10]:
from langchain.prompts import MessagesPlaceholder
MEMORY_KEY = "chat_history"
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a powerful assistant but bad at calculating lengths of word"),
        MessagesPlaceholder(variable_name = MEMORY_KEY),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name = "agent_scratchpad")
    ]
)

In [11]:
from langchain.prompts import MessagesPlaceholder

MEMORY_KEY = "chat_history"
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are very powerful assistant, but bad at calculating lengths of words.",
        ),
        MessagesPlaceholder(variable_name=MEMORY_KEY),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [13]:
from langchain_core.messages import AIMessage, HumanMessage
chat_history = []
agent = (
    {
        "input": lambda x:x["input"],
        "agent_scratchpad": lambda x:format_to_openai_tool_messages(x["intermediate_steps"]),
        "chat_history": lambda x:x["chat_history"]
    }
    |prompt
    |llm_with_tools
    |OpenAIToolsAgentOutputParser()
)
agent_executor = AgentExecutor(agent= agent, tools=tools, verbose=True)

In [14]:
input1 = "How many letters are there in the word educa"
result = agent_executor.invoke({"input": input1, "chat_history":chat_history})
chat_history.extend([
    HumanMessage(content= input1),
    AIMessage(content=result["output"])
])



> Entering new AgentExecutor chain...

Invoking: `get_word_length` with `{'word': 'educa'}`


5There are 5 words in the word "educa".

> Finished chain.


> Entering new AgentExecutor chain...

Invoking: `get_word_length` with `{'word': 'KatsukiBakugo'}`


13There are 13 letters in the word "KatsukiBakugo".

> Finished chain.


{'input': 'How many words are there in KatsukiBakugo?',
 'chat_history': [HumanMessage(content='How many words are there in the word educa'),
  AIMessage(content='There are 5 words in the word "educa".')],
 'output': 'There are 13 letters in the word "KatsukiBakugo".'}

In [16]:
input2 = input("Ask a question")
agent_executor.invoke({"input":input2, "chat_history": chat_history})
chat_history.extend([
    HumanMessage(content= input2),
    AIMessage(content=result["output"])
])



> Entering new AgentExecutor chain...

Invoking: `get_word_length` with `{'word': 'katsuki'}`


7
Invoking: `get_word_length` with `{'word': 'Bakugo'}`


6There are 2 words in "katsukiBakugo". The first word has 7 letters and the second word has 6 letters.

> Finished chain.


In [17]:
chat_history

[HumanMessage(content='How many words are there in the word educa'),
 AIMessage(content='There are 5 words in the word "educa".'),
 HumanMessage(content='How many words are there in katsukiBakugo?'),
 AIMessage(content='There are 5 words in the word "educa".')]

In [18]:
type(chat_history)

list